In [ ]:
import os
import time

# --- 1. Sirf is line ko change karein ---
# 🔴 Pehli baar 'images1_archive.tar.gz' likhein
#    Doosri baar 'images2_archive.tar.gz' likhein, etc.
ARCHIVE_FILE_NAME = "Copy of images_archive.tar.gz"

# --- 2. In paths ko hamesha same rehne dein ---
DRIVE_BASE_PATH = "/content/drive/MyDrive/images"
# Yeh hamesha same folder rahega taake images jama (collect) ho sakein
LOCAL_IMAGE_DIR = "/content/local_images"

# --- 3. Baqi code ab automatically kaam karega ---
DRIVE_TAR_PATH = os.path.join(DRIVE_BASE_PATH, ARCHIVE_FILE_NAME)
LOCAL_TAR_PATH = f"/content/{ARCHIVE_FILE_NAME}"

# Yeh line check karti hai ke folder hai ya nahi. Agar hai, to usay istemaal karti hai.
os.makedirs(LOCAL_IMAGE_DIR, exist_ok=True)
print(f"--- Setup Shuru ---")
print(f"Target Folder: {LOCAL_IMAGE_DIR}")

# --- FAST COPY ---
print(f"Copying {ARCHIVE_FILE_NAME}...")
start_time = time.time()
!cp "{DRIVE_TAR_PATH}" "{LOCAL_TAR_PATH}"
print(f"Copy complete in {(time.time() - start_time):.2f} seconds.")

# --- FAST UNPACK (ADD) ---
print(f"Extracting and ADDING images to {LOCAL_IMAGE_DIR}...")
start_time = time.time()
# Yeh command purani files ko delete nahi karta, sirf nayi files add karta hai
!tar -xzf "{LOCAL_TAR_PATH}" -C "{LOCAL_IMAGE_DIR}"
print(f"Extraction complete in {(time.time() - start_time):.2f} seconds.")

print(f"\n✅ --- READY TO TRAIN! Images are in {LOCAL_IMAGE_DIR} ---")

--- Setup Shuru ---
Target Folder: /content/local_images
Copying Copy of images_archive.tar.gz...
Copy complete in 51.17 seconds.
Extracting and ADDING images to /content/local_images...
Extraction complete in 24.03 seconds.

✅ --- READY TO TRAIN! Images are in /content/local_images ---


In [ ]:
import os

# The path where you extracted the images
LOCAL_IMAGE_DIR = "/content/local_images"

try:
    file_count = len(os.listdir(LOCAL_IMAGE_DIR))
    print(f"✅ Success! Found {file_count} images in '{LOCAL_IMAGE_DIR}'.")
except FileNotFoundError:
    print(f"❌ Error: Could not find the directory '{LOCAL_IMAGE_DIR}'.")
except Exception as e:
    print(f"An error occurred: {e}")

✅ Success! Found 30051 images in '/content/local_images'.


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm

# --- 1. Configuration (Set Your Parameters) ---
BATCH_SIZE = 32
IMG_WIDTH, IMG_HEIGHT = 300, 300

EPOCHS_FINE_TUNE = 20
# Hum purana (original) low learning rate istemaal karein ge
LEARNING_RATE_FINE_TUNE = 1e-4

# --- 2. Paths (Apne hisaab se set karein) ---
SAVED_WEIGHTS_PATH = '/content/drive/MyDrive/images/v2_finetuned_best.keras'
FINAL_MODEL_PATH = '/content/drive/MyDrive/images/v2_finetuned_best1.keras'
IMAGE_DIR = '/content/local_images'
CLEAN_CSV_PATH = '/content/drive/MyDrive/images/master_tf_clean.csv'

# --- 3. Class Weights and Names ---
CLASS_WEIGHTS_DICT = {
    0: 1.38, 1: 0.83, 2: 0.33, 3: 2.23, 4: 1.96, 5: 7.15
}
CLASS_NAMES = ['COVID-19', 'Lung Opacity', 'Normal', 'Pneumonia (Bacterial)', 'Pneumonia (Viral)', 'Tuberculosis']
NUM_CLASSES = len(CLASS_NAMES)

# --- 4. Function Definitions (Simplified) ---

def load_and_preprocess(filepath, label):
    img = tf.io.read_file(filepath)
    img = tf.image.decode_image(img, channels=3, expand_animations=False)
    img = tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])
    img = tf.keras.applications.densenet.preprocess_input(img)
    return img, label

def build_dataset(df, augment=False):
    augmentation_layers = tf.keras.Sequential([
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomRotation(0.1),
        tf.keras.layers.RandomZoom(0.1),
    ])
    dataset = tf.data.Dataset.from_tensor_slices((df['filepath'].values, df['label_idx'].values))
    dataset = dataset.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    if augment:
        dataset = dataset.map(lambda x, y: (augmentation_layers(x, training=True), y), num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

def build_densenet_model():
    base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    base_model.trainable = False
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(NUM_CLASSES, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

# --- 5. Data Loading & Pipeline Execution ---
print(f"Loading data from {CLEAN_CSV_PATH}...")
df = pd.read_csv(CLEAN_CSV_PATH)
df['filepath'] = df['filename'].apply(lambda x: os.path.join(IMAGE_DIR, x))
class_indices = {name: i for i, name in enumerate(CLASS_NAMES)}
df['label_idx'] = df['label'].map(class_indices)
train_val_df, test_df = train_test_split(df, test_size=0.1, random_state=42, stratify=df['label'])
train_df, val_df = train_test_split(train_val_df, test_size=(len(test_df)/len(train_val_df)), random_state=42, stratify=train_val_df['label'])
train_dataset = build_dataset(train_df, augment=True)
val_dataset = build_dataset(val_df)
test_dataset = build_dataset(test_df)

# --- 6. Model Build and Weight Load (Skip Stage 1) ---
print("Building DenseNet-121 model structure...")
model = build_densenet_model()

print(f"--- STAGE 1 SKIPPED ---")
print(f"Loading best weights (83.42% Peak) from '{SAVED_WEIGHTS_PATH}'...")
try:
    model.load_weights(SAVED_WEIGHTS_PATH)
    print("✅ Weights loaded successfully. Starting Fine-Tuning from 83.42%.")
except Exception as e:
    print(f"❌ ERROR: Weights load nahi huin. {e}")
    raise

# --- 7. STAGE 2: Simple Fine-Tuning Execution (PURANA CODE) ---
print("\n--- STAGE 2: Simple Fine-tuning the FULL model ---")

# (A) Index 4 par Base model ko pakrein
base_model = model.layers[4]

# (B) Poora Base Model unfreeze karein
base_model.trainable = True

print("✅ Poora Base Model (Index 4) Fine-tuning ke liye unfreeze kar diya gaya hai.")

# (C) Model ko compile karein
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE_FINE_TUNE),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# (D) Callbacks (Puranay callbacks)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)
model_checkpoint = ModelCheckpoint(FINAL_MODEL_PATH, monitor='val_loss', save_best_only=True, verbose=1)

# NAYA CALLBACK: Agar 3 epochs tak behtari na ho to LR kam karo
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7, verbose=1)

# (E) Training run karein
history_fine_tune = model.fit(
    train_dataset,
    epochs=EPOCHS_FINE_TUNE,
    validation_data=val_dataset,
    # Yahan list ko update karein
    callbacks=[early_stopping, model_checkpoint, reduce_lr],
    class_weight=CLASS_WEIGHTS_DICT
)
print("--- Simple Fine-Tuning Complete! ---")

Loading data from /content/drive/MyDrive/images/master_tf_clean.csv...
Building DenseNet-121 model structure...
29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
--- STAGE 1 SKIPPED ---
Loading best weights (83.42% Peak) from '/content/drive/MyDrive/images/v2_finetuned_best.keras'...
✅ Weights loaded successfully. Starting Fine-Tuning from 83.42%.

--- STAGE 2: Simple Fine-tuning the FULL model ---
✅ Poora Base Model (Index 4) Fine-tuning ke liye unfreeze kar diya gaya hai.
Epoch 1/20
751/751 ━━━━━━━━━━━━━━━━━━━━ 0s 774ms/step - accuracy: 0.7893 - loss: 0.4793
Epoch 1: val_loss improved from inf to 0.51810, saving model to /content/drive/MyDrive/images/v2_finetuned_best1.keras
751/751 ━━━━━━━━━━━━━━━━━━━━ 659s 829ms/step - accuracy: 0.7893 - loss: 0.4793 - val_accuracy: 0.8152 - val_loss: 0.5181 - learning_rate: 1.0000e-04
Epoch 2/20
751/751 ━━━━━━━━━━━━━━━━━━━━ 0s 757ms/step - accuracy: 0.8024 - loss: 0.4586
Epoch 2: val_loss improved from 0.51810 to 0.48591, saving model to /content

In [ ]:

print("Evaluating model on the hold-out test set...")
# Load the best model saved by ModelCheckpoint
model.load_weights("/content/drive/MyDrive/images/v2_finetuned_best1.keras")

test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# --- IMPORTANT: Generate Classification Report ---
from sklearn.metrics import classification_report, confusion_matrix

print("Generating classification report and confusion matrix...")
# Get all true labels and predictions from the test set
y_true = []
y_pred_probs = []

for images, labels in test_dataset:
    y_true.extend(labels.numpy())
    y_pred_probs.extend(model.predict(images))

y_pred = np.argmax(y_pred_probs, axis=1)

print("\n--- Confusion Matrix ---")
print(confusion_matrix(y_true, y_pred))

print("\n--- Classification Report ---")
print(classification_report(y_true, y_pred, target_names=CLASS_NAMES))

Evaluating model on the hold-out test set...
94/94 ━━━━━━━━━━━━━━━━━━━━ 19s 202ms/step - accuracy: 0.8242 - loss: 0.4648
Test Loss: 0.4554998576641083
Test Accuracy: 0.826564610004425
Generating classification report and confusion matrix...
1/1 ━━━━━━━━━━━━━━━━━━━━ 13s 13s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 243ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 253ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 257ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 300ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 344ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 328ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 268ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 322ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 289ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 317ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 290ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 372ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 339ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 238ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 225ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step
1/1 ━━━━━━━━━━━━━━━